In [ ]:
# Importing Libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout
from keras.callbacks import ModelCheckpoint
from keras.datasets import mnist
from keras.utils import to_categorical
import tensorflow as tf
from keras.models import Model

In [ ]:
# ResNet50 Model
def build_resnet50(input_shape, num_classes):
    model = Sequential([
        # Layer 1
        Conv2D(64, (7, 7), strides=(2, 2), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'),

        # Layer 2
        Conv2D(64, (1, 1), activation='relu'),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        Conv2D(256, (1, 1), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'),

        # Layer 3
        Conv2D(128, (1, 1), activation='relu'),
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        Conv2D(512, (1, 1), activation='relu'),
        BatchNormalization(),

        # Layer 4
        Conv2D(256, (1, 1), activation='relu'),
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        Conv2D(1024, (1, 1), activation='relu'),
        BatchNormalization(),

        # Layer 5
        Conv2D(512, (1, 1), activation='relu'),
        Conv2D(512, (3, 3), padding='same', activation='relu'),
        Conv2D(2048, (1, 1), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'),

        # Flatten
        Flatten(),

        # Layer 6
        Dense(2048, activation='relu'),
        Dropout(0.5),

        # Layer 7
        Dense(2048, activation='relu'),
        Dropout(0.5),

        # Output layer
        Dense(num_classes, activation='softmax')
    ])

    return model

In [ ]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess the data
input_shape = (28, 28, 1)
num_classes = 10
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

# Split the training data into training and validation sets
from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

# Build the AlexNet-like model for MNIST
model = build_resnet50(input_shape, num_classes)
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define the ModelCheckpoint callback to save the best model
checkpoint_filepath = 'resnet50.h5'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                            save_best_only=True,
                                            monitor='val_accuracy',
                                            mode='max',
                                            verbose=1)

# Fit the model to the training data with validation data and ModelCheckpoint callback
history = model.fit(train_images, train_labels, epochs=15, batch_size=128, validation_data=(val_images, val_labels),
                    callbacks=[model_checkpoint_callback])

# Define a new model without the output layer
output_layer_index = -2  # Index of the layer before the output layer
feature_extractor = Model(inputs=model.inputs, outputs=model.layers[output_layer_index].output)

# Save the model architecture and weights
feature_extractor.save('resnet50_feature_extractor.h5')

# Evaluate the best model on the test data
best_model = tf.keras.models.load_model(checkpoint_filepath)
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


11490434/11490434 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        3200      
                                                                 
 batch_normalization (Batch  (None, 14, 14, 64)        256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 7, 7, 64)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          4160      
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                         

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


422/422 [==============================] - 33s 44ms/step - loss: 1.0665 - accuracy: 0.7241 - val_loss: 6.5240 - val_accuracy: 0.1142
Epoch 2/15
421/422 [============================>.] - ETA: 0s - loss: 0.1730 - accuracy: 0.9575
Epoch 2: val_accuracy improved from 0.11417 to 0.96250, saving model to resnet50.h5
422/422 [==============================] - 16s 38ms/step - loss: 0.1730 - accuracy: 0.9575 - val_loss: 0.1744 - val_accuracy: 0.9625
Epoch 3/15
421/422 [============================>.] - ETA: 0s - loss: 0.1096 - accuracy: 0.9743
Epoch 3: val_accuracy improved from 0.96250 to 0.97350, saving model to resnet50.h5
422/422 [==============================] - 16s 39ms/step - loss: 0.1094 - accuracy: 0.9744 - val_loss: 0.1012 - val_accuracy: 0.9735
Epoch 4/15
421/422 [============================>.] - ETA: 0s - loss: 0.0778 - accuracy: 0.9817
Epoch 4: val_accuracy did not improve from 0.97350
422/422 [==============================] - 16s 37ms/step - loss: 0.0778 - accuracy: 0.9817 - v

313/313 - 3s - loss: 0.0731 - accuracy: 0.9815 - 3s/epoch - 9ms/step
Test Loss: 0.0731
Test Accuracy: 0.9815
